In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

/Users/jainilpatel/PycharmProjects/Exercise-Correction/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


## 1. Important Landmarks and Important functions

In [3]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [4]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

## 2. Describe Dataset & Split Data

In [5]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
C    8238
L    7134
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [6]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [7]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

## 3. Build Model

### 3.1. Set up

In [13]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [14]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

### 3.4. Model with 7 layers include Dropout

In [30]:
def model_7lD_builder(hp):
    # Here the hp are the hyperparameter grid search that is performed to get the best result from the 
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])

    return model

In [24]:
# def model_7lD_with_max_values():
#     # Build the model with predefined maximum hyperparameters
#     model = Sequential()
#     model.add(Dense(36, input_dim=36, activation="relu"))
#     
#     # Use maximum values directly
#     activation = 'relu'  # Maximum value choice for activation
#     layer_1 = 512  # Maximum value for layer_1 units
#     layer_2 = 512  # Maximum value for layer_2 units
#     layer_3 = 512  # Maximum value for layer_3 units
#     dropout_1 = 0.5  # Maximum value for dropout_1 rate
#     dropout_2 = 0.5  # Maximum value for dropout_2 rate
#     learning_rate = 1e-2  # Maximum value for learning rate
#     
#     model.add(Dense(units=layer_1, activation=activation))
#     model.add(Dropout(rate=dropout_1))
#     model.add(Dense(units=layer_2, activation=activation))
#     model.add(Dropout(rate=dropout_2))
#     model.add(Dense(units=layer_3, activation=activation))
#     model.add(Dense(2, activation="softmax"))
# 
#     # Compile the model
#     model.compile(optimizer=Adam(learning_rate=learning_rate), 
#                   loss="categorical_crossentropy", 
#                   metrics=["accuracy"])
#     
#     return model

In [25]:
# Here instead of doing the grid search, we are doing the hyperband search, but in general it is kind of 

In [32]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_7'
)
tuner_7lD.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 04s]
accuracy: 0.9934130311012268

Best accuracy So Far: 0.9969097971916199
Total elapsed time: 00h 01m 37s
INFO:tensorflow:Oracle triggered exit


In [33]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test))

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x3529541f0>
Layer-2: 224 units, func:  <function relu at 0x3529541f0>
Layer-3: 0 units, func: None
Layer-4: 64 units, func:  <function relu at 0x3529541f0>
Layer-5: 0 units, func: None
Layer-6: 416 units, func:  <function relu at 0x3529541f0>
Layer-7: 2 units, func:  <function softmax at 0x352950790>

Other params:
dropout_1: 0.4
dropout_2: 0.5
learning_rate: 0.001
Epoch 1/100
1230/1230 [==============================] - 1s 832us/step - loss: 0.0805 - accuracy: 0.9694 - val_loss: 0.0181 - val_accuracy: 0.9961
Epoch 2/100
1230/1230 [==============================] - 1s 793us/step - loss: 0.0301 - accuracy: 0.9927 - val_loss: 0.0199 - val_accuracy: 0.9945
Epoch 3/100
1230/1230 [==============================] - 1s 794us/step - loss: 0.0226 - accuracy: 0.9938 - val_loss: 0.0129 - val_accuracy: 0.9961
Epoch 4/100
1230/1230 [==============================] - 1s 796us/step - loss: 0.0188 - accuracy: 0.9942 - val_loss: 

In [36]:
final_models["7_layers_with_dropout"] = model_7lD

### 3.6. Describe final models

In [37]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x3529541f0>
Layer-2: 224 units, func:  <function relu at 0x3529541f0>
Layer-3: 0 units, func: None
Layer-4: 64 units, func:  <function relu at 0x3529541f0>
Layer-5: 0 units, func: None
Layer-6: 416 units, func:  <function relu at 0x3529541f0>
Layer-7: 2 units, func:  <function softmax at 0x352950790>



## 4. Model Evaluation

### 4.1. Train set evaluation

In [38]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[0.999, 0.999]","[0.999, 0.999]","[0.999, 0.999]","[[1678, 1], [2, 1394]]"


### 4.2. Test set evaluation

In [39]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 604 
Number of columns: 37

Labels: 
C    339
L    265
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [40]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [41]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers_with_dropout,"[0.765, 1.0]","[1.0, 0.608]","[0.867, 0.756]","[[339, 0], [104, 161]]"


## 5. Dumped Model

In [44]:
# Dump the best model to a pickle file
with open("./model/bicep_dp_jainil.pkl", "wb") as f:
    pickle.dump(final_models["7_layers_with_dropout"], f)

INFO:tensorflow:Assets written to: ram://d110cc89-f31f-45a8-8c97-5c24c571421c/assets


In [45]:
# Dump final results
with open("./model/all_models_jainil.pkl", "wb") as f:
    pickle.dump(final_models, f)

INFO:tensorflow:Assets written to: ram://e52de510-44fc-4e2d-b169-4c5d9550eef3/assets
